<a href="https://colab.research.google.com/github/tutsilianna/Automatic_Text_Processing_and_Image_Processing/blob/main/Information%20Retrieval/Task_2_%7C_Information_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval

Let's download the classical data set, i.e. the CRANFIELD text set on aeronautics

In [1]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


We take queries only (we will consider queries as documents)

In [2]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


We combine  multi-string queries into one

In [3]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Let's output the couple of documents as an example

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Let's make queries to our documents

In [5]:
QUERIES = ['theory of bending', 'aeroelastic effects']


## Boolean retrieval
Let's represent each document as a "bitmask": that is a vector with a dimensionality equal to the vocabulary size, which has 1 at every position if the document contains the corresponding term; and 0 if it does not

In [6]:
# in different versions the answer could also differ, therefore it's important to have the same version
! pip install -q scikit-learn==1.4.2 # 0.22.2.post1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 22.5 MB/s eta 0:00:00


In [7]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Let's look at the representation of the first sentence

In [8]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

It's fine.

## Task 0

Now for each query from `QUERIES` let's find the nearest document from `query_data` according to the Jaccard similarity index. There are more effictive solutions to do it, but your task is to realize the algorithm computing the Jaccard index and then apply it to our data.

In [19]:
import numpy as np

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
    """
        Similarity or Jaccard similarity index: the ratio of the intersection cardinality to the union cardinality
    """
    intersection = np.logical_and(vector_a, vector_b)
    union = np.logical_or(vector_a, vector_b)
    similarity = intersection.sum() / float(union.sum())

    return similarity

#Check that the function works correctly
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

## Task 1
Now using the code below find the most similar documents for each query.

In [20]:
for q_id, query in enumerate(encoded_queries):
  # bring to the required datatype
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # calculate the Jaccard index
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # sort according to it
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)

  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # output 3 most similar documents for each query
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))



Q: theory of bending
FOUND:
    42	0.20	what are the details of the rigorous kinetic theory of gases . 
    43	0.20	(chapman-enskog theory) . 
    146	0.19	does a membrane theory exist by which the behaviour of pressurized membrane cylinders in bending can be predicted . 
Q: aeroelastic effects
FOUND:
    196	0.14	the problem of similarity for representative investigation of aeroelastic effects in a flow with the absence of heating effects . 
    204	0.12	do viscous effects seriously modify pressure distributions . 
    114	0.12	is the problem of similarity for representative investigations of aeroelastic effects in heated flow as intractable as previous investigations imply . 


We see that some texts intersecting with the query only in insignificant terms have a high Jaccard index (that is our ranking function).

# VSM

Now we are going to do similar calculations, but using tf-idf and cosine distance. To practice we make everything "manually", but in "real life" it's better to use existing effective solutions, e.g., cosine distance from scipy library.

In [21]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Advice: we highly recommend to check what tf-idf vectorizer
# is able to do, and change its parameters

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Task 2
Realize the cosine distance computation

In [25]:
import numpy as np

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Cosine distance is 1 minus the ratio of the dot product
    and the product of L2-norm (hint: there are such norms in numpy)
  """
  return 1 - (sum(vector_a * vector_b)) / (np.linalg.norm(vector_a, 2) * np.linalg.norm(vector_b, 2))

  return
#Check that the function is working correctly
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5

[0 0 1 0 0]



Now let's find the nearset documents to the query according to the cosine distance between the document vector and the query representation

In [26]:
for q_id, query in enumerate(tfidf_encoded_queries):

  # bring to the required datatype
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Cosine distance
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # sort according to it
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)

  print("Q: %s\nFOUND:" % QUERIES[q_id])

  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Выходные данные были обрезаны до нескольких последних строк (5000).
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

<ipython-input-25-693db63d0360>:9: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - (sum(vector_a * vector_b)) / (np.linalg.norm(vector_a, 2) * np.linalg.norm(vector_b, 2))


Выходные данные были обрезаны до нескольких последних строк (5000).
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

## Individual task


Using queries to texts from the CRANFIELD text set on aeronautics (the same is used in the example) as documents, find the pair of documents that are the most similar to the specified query.

The text of the query: **supersonic cascade**.

1. Enter the index of the most similar document according to the Jaccard similarity index
2. Enter the value of the Jaccard similarity index of the most similar document
3. Enter the index of the second most similar document according to the Jaccard similarity index
4. Enter the value of the Jaccard similarity index of the second most similar document

The text of the query: **swept cylinders**.

1. Enter the index of the most similar document according to the cosine distance
2. Enter the value of the cosine distance for the most similar document
3. Enter the index of the second most similar document according to the cosine distance
4. Enter the value of the cosine distance for the second most similar document